In [1]:
from restaurant.models import Category, Restaurant, Item

In [10]:
카테고리_치킨 = Category.objects.get(name='치킨')

In [62]:
import json

In [12]:
with open('../crawler/restaurant_list.json', 'rt', encoding='utf8') as f:
    json_string = f.read()
    restaurant_list = json.loads(json_string)
    
len(restaurant_list)

20

In [13]:
for restaurant in restaurant_list:
    print(restaurant.keys())

dict_keys(['rating', 'subtitle', 'additional_discount_currently_active', 'free_delivery_threshold', 'delivery_fee_explanation', 'is_available_delivery', 'threshold', 'adv_distance', 'open', 'discount_until', 'central_billing', 'city', 'review_reply_count', 'sms_bonus', 'franchise_id', 'phone_downlisted', 'franchise_name', 'review_image_count', 'top100_restaurant', 'is_available_pickup', 'additional_discount_per_menu', 'delivery_fee', 'additional_discount_pickup', 'phone', 'additional_discount_only_for_user', 'has_terminal', 'categories', 'min_order_amount', 'distance', 'logo_url', 'except_cash', 'name', 'section_pos', 'relay_methods', 'one_dish', 'discount_percent', 'advertisement_type', 'phone_order', 'dine_in', 'advertisement_rank', 'thumbnail_message', 'additional_discount_pickup_per_menu', 'open_time_description', 'keywords', 'new_rating', 'app_order', 'premium', 'lng', 'list_pos', 'id', 'can_review', 'estimated_delivery_time', 'additional_discount_pickup_only_for_user', 'review_co

In [17]:
obj_list = []

for meta in restaurant_list:
#     print('{name} | {address} | 평점 : {rating}'.format(**restaurant))
    name = meta['name']
    latlng = '{lat},{lng}'.format(**meta)
    restaurant = Restaurant(category=카테고리_치킨, name=name, desc='', latlng=latlng, meta=meta)
    obj_list.append(restaurant)

In [20]:
Restaurant.objects.bulk_create(obj_list)

[<Restaurant: 원할머니보쌈-성수점>,
 <Restaurant: 미스터피자-성수점>,
 <Restaurant: 배스킨라빈스-성수역점>,
 <Restaurant: 놀부부대찌개-시그마밸리점>,
 <Restaurant: 나뚜루팝-건대스타시티점>,
 <Restaurant: 파파존스-성수점>,
 <Restaurant: 파리바게뜨-뚝도시장점>,
 <Restaurant: 피자헛-뚝섬2호점>,
 <Restaurant: 청하루>,
 <Restaurant: 동강>,
 <Restaurant: 버거킹-건대화양점>,
 <Restaurant: 육회공작소-건대점>,
 <Restaurant: 원할머니보쌈-성수점>,
 <Restaurant: 육회한연어-건대점>,
 <Restaurant: 피자마스터>,
 <Restaurant: KFC-세종대점>,
 <Restaurant: 미스고육개장-건대점>,
 <Restaurant: 25시서울야식-왕십리점>,
 <Restaurant: 요리왕>,
 <Restaurant: 신강투루판양꼬치>]

In [22]:
from urllib.parse import quote

In [ ]:
quote()

In [23]:
# from selenium import webdriver
# from bs4 import BeautifulSouplyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

# driver = webdriver.PhantomJS() # 브라우져를 phantomjs 을 사용함. ie, firefox, chrome 도 가능함
# driver.get('https://www.yogiyo.co.kr/mobile/#/서울/133123/')
# bs = BeautifulSoup(driver.page_source, 'html5lib')
# print(bs.findAll("div"))

## --------------------------------------------------------------------------------------

In [1]:
import os
import json
import requests
from django.core.files.base import ContentFile
from restaurant.models import Category, Restaurant, Item
from urllib.parse import urljoin
import copy
# from fake_useragent import UserAgent

In [2]:
# 식당 목록 페이지의 식당 Request URL
# https://www.yogiyo.co.kr/api/v1/restaurants-geo/?items=20&lat=37.4910792&lng=127.1253318&order=rank&page=0&search=&zip_code=138169
# 해당 식당 메뉴 페이지의 메뉴 Request URL
# https://www.yogiyo.co.kr/api/v1/restaurants/294835/menu/?add_photo_menu=android
# 해당 식당 메뉴 페이지의 리뷰 Request URL
# https://www.yogiyo.co.kr/api/v1/reviews/294835/?count=10&only_photo_review=false&page=1&sort=time
class Crawler:
    BASE_URL = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
        
    def get_restaurant_list(self, zip_code):
        url = self.BASE_URL + '/api/v1/restaurants-geo/'
        params = dict(
            items=20,
            order='rank',
            page=0,
            search='',
            zip_code=zip_code,
        )
        response = requests.get(url, params=params, headers=self.headers)
        return response.json()
    
    def get_menu_list(self, restaurant_id):
        menu_url = self.BASE_URL + f'/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        div_url = menu_url.format(restaurant_id=restaurant_id)
        response = requests.get(div_url, headers=self.headers)
        return response.json()

In [3]:
headers = {
    'X-ApiKey': 'iphoneap',
    'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
    'X-MOD-SBB-CTYPE': 'xhr',
}

In [4]:
headers

{'X-ApiKey': 'iphoneap',
 'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
 'X-MOD-SBB-CTYPE': 'xhr'}

In [5]:
yogiyo = Crawler(headers)

In [7]:
zip_code = '138200'
restaurants = yogiyo.get_restaurant_list(zip_code)['restaurants']
restaurants

[{'rating': 200013.050694638,
  'subtitle': '',
  'additional_discount_currently_active': True,
  'free_delivery_threshold': 0,
  'delivery_fee_explanation': '배달 요금 4000원, 최소 주문 금액 15000원',
  'is_available_delivery': True,
  'threshold': 8000,
  'adv_distance': None,
  'open': True,
  'discount_until': None,
  'central_billing': False,
  'city': '서울',
  'review_reply_count': 6,
  'sms_bonus': False,
  'franchise_id': 575,
  'phone_downlisted': False,
  'franchise_name': '씨제이푸드빌(주) / 계절밥상',
  'review_image_count': 6,
  'top100_restaurant': False,
  'is_available_pickup': False,
  'additional_discount_per_menu': 0,
  'delivery_fee': 4000,
  'additional_discount_pickup': 0,
  'phone': '0000000000',
  'additional_discount_only_for_user': True,
  'has_terminal': False,
  'categories': ['한식', '프랜차이즈'],
  'min_order_amount': 15000,
  'distance': 0.5378802086566781,
  'logo_url': '/media/franchise_logos/계절밥상_20180814_Franchise이미지약정서_crop_200x200_FUWjH8h.jpg',
  'except_cash': True,
  'name': '

In [25]:
for rest in restaurants:
    categories = rest['categories']
#     print(category_name)
#     category = Category.objects.get(name=category_name)
    latlng = '{lat},{lng}'.format(**rest)
    logo_url = urljoin(Crawler.BASE_URL, rest['logo_url'])
    restaurant = Restaurant.objects.create(name=rest['name'], latlng=latlng, meta=rest)
    
    for category_name in categories:
        category = Category.objects.get_or_create(name=category_name)[0]
        restaurant.category.add(category)
    
    print(category)
    logo_name = os.path.basename(logo_url)
    logo_data = requests.get(logo_url).content
    restaurant.photo.save(logo_name, ContentFile(logo_data), save=False)
    print('{name}: {lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
    restaurant.save()

프랜차이즈
계절밥상-문정점: 37.4806561039041,127.123703577866, ['한식', '프랜차이즈'], /media/franchise_logos/계절밥상_20180814_Franchise이미지약정서_crop_200x200_FUWjH8h.jpg
프랜차이즈
뉴욕버거-가든파이브점: 37.4783224971508,127.119519301344, ['피자양식', '프랜차이즈'], /media/restaurant_logos/뉴욕버거_20171027_Franchise이미지약정서_crop_200x200_eSBGjty.jpg
프랜차이즈
멕시카나-송파점: 37.5035242254289,127.113196978845, ['치킨', '프랜차이즈'], /media/restaurant_logos/멕시카나치킨_20171115_Franchise이미지약정서_crop_200x200_JRolZH1.jpg
예약픽업
파리바게뜨-문정컬처밸리점: 37.4846585667716,127.120950910753, ['프랜차이즈', '카페디저트', '예약픽업'], /media/franchise_logos/파리바게뜨_20180712_Franchise이미지약정서_crop_200x200_s6Zm07l.jpg
치킨
또래오래-송파문정점: 37.4837877,127.1299678, ['프랜차이즈', '치킨'], /media/franchise_logos/또래오래_20151126_Franchise이미지약정서_crop_200x200_89SjZAD.jpg
피자양식
뽕뜨락피자-문정점: 37.4815143999305,127.12584, ['프랜차이즈', '피자양식'], /media/restaurant_logos/뽕뜨락피자_20170706_Franchise이미지약정서_crop_200x200_93UZMUg.jpg
프랜차이즈
김준현의피자헤븐-잠실점: 37.5097126616517,127.11769381551, ['피자양식', '프랜차이즈'], /media/franchise_logos/피자헤븐_20161011_Fran

In [10]:
# for rest in restaurants:
#     category_name = rest['categories'][0]
#     print(category_name)
#     category = Category.objects.get(name=category_name)
#     latlng = '{lat},{lng}'.format(**rest)
#     logo_url = urljoin(Crawler.BASE_URL, rest['logo_url'])
#     restaurant = Restaurant(category=category, name=rest['name'], latlng=latlng, meta=rest)
    
#     logo_name = os.path.basename(logo_url)
#     logo_data = requests.get(logo_url).content
#     restaurant.photo.save(logo_name, ContentFile(logo_data), save=False)
#     print('{name}: {lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
#     restaurant.save()

피자양식
뉴욕버거-가든파이브점: 37.4783224971508,127.119519301344, ['피자양식', '프랜차이즈'], /media/restaurant_logos/뉴욕버거_20171027_Franchise이미지약정서_crop_200x200_eSBGjty.jpg
치킨
페리카나-문정점: 37.4872785,127.1336498, ['치킨', '프랜차이즈'], /media/franchise_logos/페리카나_20160321_Franchise이미지약정서_crop_200x200.jpg
카페디저트
파리바게뜨-문정컬처밸리점: 37.4846585667716,127.120950910753, ['카페디저트', '프랜차이즈', '예약픽업'], /media/franchise_logos/파리바게뜨_20180712_Franchise이미지약정서_crop_200x200_s6Zm07l.jpg
프랜차이즈
또래오래-송파문정점: 37.4837877,127.1299678, ['프랜차이즈', '치킨'], /media/franchise_logos/또래오래_20151126_Franchise이미지약정서_crop_200x200_89SjZAD.jpg
프랜차이즈
호식이두마리치킨-문정점: 37.4883625918614,127.133728849877, ['프랜차이즈', '야식', '치킨'], /media/restaurant_logos/호식이두마리치킨_20180212_Franchise이미지약정서_crop_200x200_jP6Bc6j.PNG
프랜차이즈
뽕뜨락피자-문정점: 37.4815143999305,127.12584, ['프랜차이즈', '피자양식'], /media/restaurant_logos/뽕뜨락피자_20170706_Franchise이미지약정서_crop_200x200_93UZMUg.jpg
한식
스쿨푸드-문정딜리버리점: 37.4944886331396,127.126751572502, ['한식', '분식', '프랜차이즈'], /media/franchise_logos/스쿨푸드_20180718_Franchis

In [12]:
# cg_name = copy.deepcopy(rest['categories'])
# # cg_name = cg_name.split(",")
# # webuser.benefits.add(*Used_benefit.objects.filter(extid__in=productlist))
# Restaurant?


In [13]:
# print(type(cg_name))

In [14]:
# ast

In [15]:
# category_name = rest['categories']
# print(category_name)
# print(type(category_name))
# # category = Category.objects.get(name=category_name)

# print(category)

In [16]:
# for rest in restaurants:
#     category_name = rest['categories'][0]
#     print(category_name)
#     category = Category.objects.get(name=category_name)
#     latlng = '{lat},{lng}'.format(**rest)
#     logo_url = urljoin(Crawler.BASE_URL, rest['logo_url'])
#     restaurant = Restaurant(category=category, name=rest['name'], latlng=latlng, meta=rest)
    
#     logo_name = os.path.basename(logo_url)
#     logo_data = requests.get(logo_url).content
#     restaurant.photo.save(logo_name, ContentFile(logo_data), save=False)
#     print('{name}: {lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
#     restaurant.save()

In [17]:
# Restaurant.objects.all()

In [18]:
# for restaurant in Restaurant.objects.all():
#     print(restaurant.name,'|', restaurant.latlng,'|', restaurant.meta['id'])
################################################################################!#!
# for restaurant in Restaurant.objects.all():
#     print(restaurant.name, restaurant.category)
#     print(restaurant.category.all())

print(type(restaurant.category.all()))


<class 'django.db.models.query.QuerySet'>


In [1]:
for restaurant in Restaurant.objects.all():
    restaurant_id = restaurant.meta['id']
    menu_list = yogiyo.get_menu_list(restaurant_id)
    print(restaurant.name)
    for sub_menu_list in menu_list:
        items = sub_menu_list['items']
        for item_meta in items:
            item = Item(restaurant=restaurant, name=item_meta['name'], price=item_meta['price'], meta=item_meta)
            
            item_image_url = item_meta.get('image', '')
            if item_image_url:
                item_image_url = urljoin(Crawler.BASE_URL, item_meta['image'])
                
                item_image_name = os.path.basename(item_image_url)
                item_image_data = requests.get(item_image_url).content
                item.photo.save(item_image_name, ContentFile(item_image_data), save=False)
                zip_code
            print('restaurant name : {}'.format(item.name))
            item.save()